# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [27]:
# your code here
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


In [28]:
df = pd.read_csv('/home/juan/IronHack/dataptmad0420/module-3/deep-learning/your-code/tic-tac-toe.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [29]:
df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [30]:
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(df)

In [31]:
pd.DataFrame(X_encoded.todense()).head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [32]:
from sklearn.preprocessing import LabelEncoder 

cols = list(df.columns)


le = LabelEncoder() 

for col in cols[:-1]:
    df[col] = le.fit_transform(df[col]) 

In [33]:
df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,True
1,2,2,2,2,1,1,1,2,1,True
2,2,2,2,2,1,1,1,1,2,True
3,2,2,2,2,1,1,1,0,0,True
4,2,2,2,2,1,1,0,1,0,True
...,...,...,...,...,...,...,...,...,...,...
953,1,2,2,2,1,1,1,2,2,False
954,1,2,1,2,2,1,2,1,2,False
955,1,2,1,2,1,2,2,1,2,False
956,1,2,1,1,2,2,2,1,2,False


In [34]:
encoder = LabelEncoder()
encoded_df = df.apply(encoder.fit_transform)
encoded_df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
953,1,2,2,2,1,1,1,2,2,0
954,1,2,1,2,2,1,2,1,2,0
955,1,2,1,2,1,2,2,1,2,0
956,1,2,1,1,2,2,2,1,2,0


In [35]:
df_input = encoded_df[['TL','TM','TR','ML','MM','MR','BL','BM','BR']]
df_output = encoded_df['class']

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [36]:
from sklearn.model_selection import train_test_split
df_input_train, df_input_test, df_output_train, df_output_test = train_test_split(df_input, df_output)
display(df_input_train.shape)
display(df_output_train.shape)

(718, 9)

(718,)

In [104]:
import tensorflow as tf
model = tf.keras.Sequential()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(2,activation='softmax')
])
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
df_input_train = pd.DataFrame(df_input_train)
df_output_train = pd.DataFrame(df_output_train)

In [105]:
mod = model.fit(df_input_train, df_output_train, epochs=400, batch_size=64, validation_data=(df_input_train, df_output_train))

Epoch 1/400
12/12 [==============================] - 0s 13ms/step - loss: 0.6493 - accuracy: 0.6532 - val_loss: 0.6363 - val_accuracy: 0.6602
Epoch 2/400
12/12 [==============================] - 0s 4ms/step - loss: 0.6312 - accuracy: 0.6602 - val_loss: 0.6212 - val_accuracy: 0.6602
Epoch 3/400
12/12 [==============================] - 0s 5ms/step - loss: 0.6163 - accuracy: 0.6616 - val_loss: 0.6062 - val_accuracy: 0.6894
Epoch 4/400
12/12 [==============================] - 0s 5ms/step - loss: 0.6041 - accuracy: 0.7019 - val_loss: 0.5942 - val_accuracy: 0.7089
Epoch 5/400
12/12 [==============================] - 0s 5ms/step - loss: 0.5924 - accuracy: 0.6992 - val_loss: 0.5823 - val_accuracy: 0.7298
Epoch 6/400
12/12 [==============================] - 0s 5ms/step - loss: 0.5796 - accuracy: 0.7354 - val_loss: 0.5708 - val_accuracy: 0.7423
Epoch 7/400
12/12 [==============================] - 0s 5ms/step - loss: 0.5683 - accuracy: 0.7521 - val_loss: 0.5600 - val_accuracy: 0.7563
Epoch 8/400


In [106]:
val_loss, val_acc = model.evaluate(df_input_train, df_output_train)
print(val_loss, val_acc)

23/23 [==============================] - 0s 668us/step - loss: 0.3231 - accuracy: 0.9903
0.3231230080127716 0.9902507066726685


In [107]:
model.save('tic-tac-toe.model')

INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [108]:
# your code here

new_model = tf.keras.models.load_model('tic-tac-toe.model')

In [109]:
predictions = new_model.predict(df_input_test)

In [110]:
#print(predictions)
import numpy as np
print(np.argmax(predictions[4]))

0


In [111]:
val_loss, val_acc = model.evaluate(df_input_test, df_output_test)
print(val_loss, val_acc)

8/8 [==============================] - 0s 1ms/step - loss: 0.3999 - accuracy: 0.9125
0.3998965322971344 0.9125000238418579


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [103]:
# your code here
tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
'''
The most helpful thing to improve my model has been increase the number of epochs

'''